In [5]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [6]:
data=pd.read_csv("original datasets/train.csv")


In [7]:
imputer=IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10), max_iter=100)

In [8]:
fullrows = data.loc[data.notna().sum(axis=1)==79, data.columns!= 'galaxy']
imputer.fit(fullrows)
data_without_galaxy = data.loc[:, data.columns!= 'galaxy']
imp_d=imputer.transform(data_without_galaxy)
data_transformed=pd.DataFrame(imp_d, columns=data_without_galaxy.columns)
data_transformed.insert(1, 'galaxy', data.galaxy)

C:\Users\Yassawe\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:670: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [9]:
data_transformed.head(100)

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
0,990025.0,Large Magellanic Cloud (LMC),0.628657,63.125200,27109.234310,0.646039,8.240543,7.242049,0.632596,0.504113,192.417996,74.553265,62.672917,38.305483,681.841086,22.131436,150.259290,129.841789,12.403812,490.446

In [10]:
d_shuffle = data_transformed.sample(frac=1)

In [11]:
train = d_shuffle.iloc[0:2532 ,:]
test = d_shuffle.iloc[2532:,:]

In [12]:
x_train=train.loc[:, train.columns!='y']
y_train=train.loc[:, train.columns=='y']
x_test=test.loc[:, test.columns!='y']
y_test=test.loc[:, test.columns=='y']


In [15]:
x_train.to_csv('datasets/iterative_imputer/x_train.csv')
y_train.to_csv('datasets/iterative_imputer/y_train.csv')
x_test.to_csv('datasets/iterative_imputer/x_test.csv')
y_test.to_csv('datasets/iterative_imputer/y_test.csv')

In [ ]:
"""
galaxies=x.galaxy.unique().tolist()
frames=[]
for galaxy in galaxies:
    df = x.loc[x.galaxy==galaxy, x.columns!= 'galaxy']
    indexes = df.notna().sum(axis=1).nlargest(3).index
    da = df.loc[indexes, :]
    imputer.fit(da)
    imp_df = imputer.transform(df)
    imp_df = pd.DataFrame(imp_df)
    imp_df.insert(1, 'galaxy', galaxy)
    frames.append(imp_df)     
"""

In [ ]:
transformed_x = pd.concat(frames)

In [ ]:
transformed_x.head(30)

In [ ]:
print(transformed_x.shape)
print(y.shape)